In [321]:
def probAttr(data,attr,val):
    Total=data.shape[0]
    from collections import Counter
    cnt=Counter(x for x in data[attr])
    return cnt[val],cnt[val]/Total

In [322]:
def probAttrConcept(data,attr,val,concept,cVal,countConcept):
#     print(attr,val)
    count={}
    prob={}
    C = data[concept]
    A = data[attr]
    for a in range(len(data[attr])):
#         print(A[a],C[a])
        for v in val:
            if(A[a]==v and C[a]==cVal):
                if v not in count:
                    count[v]=1
                else:
                    count[v]=count[v]+1
    for a in count:
        prob[a] = count[a]/countConcept
    return prob

In [390]:
def train(data,AttributeList,concept):
    Attr={}
    probability_list={}
    for a in AttributeList:
        Attr[a] = list(set(data[a]))
        
    conceptVals = list(set(data[concept]))
    conceptProbs = {}
    countConcept={}
    AttrConcept = {}
    for cVal in conceptVals:
        countConcept[cVal],conceptProbs[cVal] = probAttr(data,concept,cVal)
    
    for val in Attr:
        probability_list[val]={}
        AttrConcept[val] = {}
        for v in Attr[val]:
            a,probability_list[val][v]=probAttr(data,val,v)
        for cVal in conceptVals:
            AttrConcept[val][cVal]=probAttrConcept(data,val,Attr[val],concept,cVal,countConcept[cVal])
        
    print("P(A) : ",conceptProbs,"\n")
    print("P(X/A) : ",AttrConcept,"\n")
    print("P(X) : ",probability_list,"\n")
    return conceptProbs,AttrConcept,probability_list


In [391]:
def test(examples,AttributeList,conceptProbs,AttrConcept,probability_list,data,concept_list,Total):
    misclassification_count=0
#     Total = len(examples)-1
    for ex in range(1,len(examples)):
        px={}
        for x in range(1,len(examples[ex])-1):
            for a in AttributeList:
                for c in concept_list:
                    if examples[ex][x] in AttrConcept[a][c]:  
                        if c not in px:
                            px[c] = 1*AttrConcept[a][c][examples[ex][x]]
                        else:
                            px[c] = px[c]*AttrConcept[a][c][examples[ex][x]]
        classification = max(px,key=px.get)
        if(classification!=examples[ex][-1]):
            misclassification_count+=1
    misclassification_rate=misclassification_count*100/Total
    accuracy=100-misclassification_rate
    print("Misclassification Count={}".format(misclassification_count))
    print("Misclassification Rate={}%".format(misclassification_rate))
    print("Accuracy={}%".format(accuracy))

In [392]:
def main():
    import pandas as pd
    from pandas import DataFrame 
    from collections import Counter
    data = DataFrame.from_csv('data_train.csv')
#     print("\n Given Play Tennis Data Set:\n\n", data)
    AttributeList=list(data)[:-1]
    num_of_attributes=len(AttributeList)
    concept=str(list(data)[-1])
    Total=data.shape[0]
    conceptProbs,AttrConcept,probability_list = train(data,AttributeList,concept)

    import csv
    with open('data_test.csv')  as csvFile:
        examples = [list(line) for line in csv.reader(csvFile)]
#     print(examples)
    concept_list =list(Counter(x for x in data[concept]))
    test(examples,AttributeList,conceptProbs,AttrConcept,probability_list,data,concept_list,Total)

In [393]:
main()

P(A) :  {'No': 0.4, 'Yes': 0.6} 

P(X/A) :  {'classTime': {'No': {'Afternoon': 0.6071428571428571, 'Morning': 0.10714285714285714, 'Weekend': 0.2857142857142857}, 'Yes': {'Morning': 0.6190476190476191, 'Weekend': 0.09523809523809523, 'Afternoon': 0.2857142857142857}}, 'subjectType': {'No': {'Theory': 0.5357142857142857, 'Problematic': 0.03571428571428571, 'Practicle': 0.07142857142857142, 'Programing': 0.35714285714285715}, 'Yes': {'Practicle': 0.30952380952380953, 'Programing': 0.2857142857142857, 'Theory': 0.19047619047619047, 'Problematic': 0.21428571428571427}}, 'hungry': {'No': {'No': 0.7857142857142857, 'Yes': 0.21428571428571427}, 'Yes': {'No': 0.9285714285714286, 'Yes': 0.07142857142857142}}, 'weather': {'No': {'Hot': 0.5, 'Mild': 0.2857142857142857, 'Cool': 0.21428571428571427}, 'Yes': {'Cool': 0.38095238095238093, 'Hot': 0.3333333333333333, 'Mild': 0.2857142857142857}}} 

P(X) :  {'classTime': {'Weekend': 0.17142857142857143, 'Afternoon': 0.4142857142857143, 'Morning': 0.4142